In [ ]:
base_folder_best_egg =  '/d/shared/silver_projects_v2/bestegg/personalloanv1/modeling/model_artifacts/personalloan/exp7.6.CO36m.pre_test.all_features_foldinvalid/'
base_folder_patel_co =  '/d/shared/silver_projects_v2/patelcocu/autoplccv1/modeling/auxiliary_model_artifacts/personalloan/model1Personalloan/'

In [ ]:
import os
import pandas as 

root_path = "/d/shared/silver_projects_v2/bestegg/personalloanv1/modeling/model_artifacts/personalloan/"

lgd_folders = []

for dirpath, dirnames, filenames in os.walk(root_path):
    for dirname in dirnames:
        if "lgd" in dirname or "LGD" in dirname: 
            full_path = os.path.join(dirpath, dirname)
            lgd_folders.append(full_path)

for folder in lgd_folders:
    print(folder)

print(f"\nTotal LGD folders found: {len(lgd_folders)}")


/d/shared/silver_projects_v2/bestegg/personalloanv1/modeling/model_artifacts/personalloan/lgd_model
/d/shared/silver_projects_v2/bestegg/personalloanv1/modeling/model_artifacts/personalloan/lgd_model/NationalLGDexp1
/d/shared/silver_projects_v2/bestegg/personalloanv1/modeling/model_artifacts/personalloan/lgd_model/NationalLGDexp2_eqf

Total LGD folders found: 3


In [56]:
ls /d/shared/silver_projects_v2/bestegg/personalloanv1/modeling/model_artifacts/personalloan/

co_exp_4_3_posttest/
co_exp_4_3_pretest/
exp3_9_fixplaceholders_fixasset_redo/
exp6.2.CO.post_test.all_features_foldinvalid/
exp6.2.CO.pre_test.all_features_foldinvalid/
exp6.4.CO.20172018.all_features_foldinvalid/
exp6.4.CO.post_test.all_features_foldinvalid/
exp6.4.CO.pre_test.all_features_foldinvalid/
exp6.7.CO.20172018.all_features_with2017_new_weights_foldinvalid/
exp6.7.CO.post_test.all_features_with2017_new_weights_foldinvalid/
exp6.7.CO.pre_test.all_features_with2017_new_weights_foldinvalid/
exp6.8.CO.20172018.all_features_no2017_new_weights_foldinvalid/
exp6.8.CO.post_test.all_features_no2017_new_weights_foldinvalid/
exp6.8.CO.pre_test.all_features_no2017_new_weights_foldinvalid/
exp7.2.CO36m.post_test.all_features_foldinvalid/
exp7.2.CO36m.pre_test.all_features_foldinvalid/
exp7.2.CO36m.pre_test.all_features_foldinvalid_1718/
exp7.3.CO36m.20172018.all_features_foldinvalid/
exp7.3.CO36m.post_test.all_features_foldinvalid/
exp7.3.CO36m.pre_test.all_features_foldinvalid/
exp7.4.

In [57]:
import pickle
import model_engine
import warnings
import zaml
import zamlmodel
warnings.filterwarnings('ignore')
def load_pipe(folder, mrm = False):
    if mrm:
        location = f'{folder}mrm_pipeline.obj'
    else:
        location = f'{folder}pipeline.obj'
    print(f'MRM: {mrm} location: {location}')
    with open(location, 'rb') as f:
        meta_data = pickle.load(f)
        pipe = pickle.load(f)
    return pipe

In [29]:
init_transformers[i].__dict__

{'_zaml_ver': '34.2.0',
 '_other_params': {},
 'name': 'client_data FE',
 'inbounds': ['client_data'],
 'outbounds': ['Merge data'],
 'transformer': <model_engine.feature_engine.engines.one_to_one_engine.OneToOneEngine at 0x7ff5a8cf2920>,
 'data': None,
 'valid_data': None,
 'feature_types': None,
 'data_cached': None,
 'fit_params': None,
 'is_run': False}

In [32]:
client_dict = {
    'patel_co': base_folder_patel_co,
    'bestegg': base_folder_best_egg
}

In [24]:
client_dict = {
    'patel_co': base_folder_patel_co,
    'bestegg': base_folder_best_egg
}

for client, folder in client_dict.items():
    print(f'\n==============================')
    print(f'CLIENT: {client}')
    print(f'==============================')

    keys_in_reg = set()
    tf_key_map_reg = {}

    for mrm in [True, False]:

        pipe = load_pipe(folder=folder, mrm=mrm)
        pipe_dict = pipe.__dict__
        dict_keys = set(pipe_dict.keys())

        # ----------------------------
        # PIPE-LEVEL KEY COMPARISON
        # ----------------------------
        if mrm:
            keys_in_reg = dict_keys
        else:
            missing_in_no_mrm = keys_in_reg - dict_keys
            new_in_no_mrm = dict_keys - keys_in_reg

            print('\n--- PIPE DICT KEY DIFFS (no-mrm vs reg) ---')
            print('Missing in no-mrm:', sorted(missing_in_no_mrm))
            print('New in no-mrm:', sorted(new_in_no_mrm))

        # ----------------------------
        # TRANSFORMER COLLECTION
        # ----------------------------
        init_transformers = pipe_dict.get('init_transformers', [])

        tf_key_map = {}
        for t in init_transformers:
            d = t.__dict__
            name = d.get('name')
            if name is not None:
                tf_key_map[name] = set(d.keys())

        # ----------------------------
        # TRANSFORMER COMPARISON
        # ----------------------------
        if mrm:
            tf_key_map_reg = tf_key_map
        else:
            tf_key_map_no_mrm = tf_key_map

            reg_names = set(tf_key_map_reg.keys())
            no_mrm_names = set(tf_key_map_no_mrm.keys())

            only_reg = reg_names - no_mrm_names
            only_no_mrm = no_mrm_names - reg_names
            common = reg_names & no_mrm_names

            print('\n--- TRANSFORMER PRESENCE (no-mrm vs reg) ---')
            print('Missing in no-mrm:', sorted(only_reg))
            print('New in no-mrm:', sorted(only_no_mrm))

            print('\n--- TRANSFORMER INTERNAL KEY DIFFS ---')
            for name in sorted(common):
                reg_keys = tf_key_map_reg[name]
                no_mrm_keys = tf_key_map_no_mrm[name]

                missing_keys = reg_keys - no_mrm_keys
                new_keys = no_mrm_keys - reg_keys

                if not missing_keys and not new_keys:
                    print(f'{name}: SAME')
                else:
                    print(f'{name}:')
                    print('  Missing keys in no-mrm:', sorted(missing_keys))
                    print('  New keys in no-mrm:', sorted(new_keys))



CLIENT: patel_co
MRM: True location: /d/shared/silver_projects_v2/patelcocu/autoplccv1/modeling/auxiliary_model_artifacts/personalloan/model1Personalloan/mrm_pipeline.obj
MRM: False location: /d/shared/silver_projects_v2/patelcocu/autoplccv1/modeling/auxiliary_model_artifacts/personalloan/model1Personalloan/pipeline.obj

--- PIPE DICT KEY DIFFS (no-mrm vs reg) ---
Missing in no-mrm: []
New in no-mrm: ['_is_binary', 'classes_']

--- TRANSFORMER PRESENCE (no-mrm vs reg) ---
Missing in no-mrm: []
New in no-mrm: []

--- TRANSFORMER INTERNAL KEY DIFFS ---
Merge data: SAME
app: SAME
app FE: SAME
bnkr: SAME
bnkr FE: SAME
client_data: SAME
client_data FE: SAME
collec: SAME
collec FE: SAME
config: SAME
config FE: SAME
inq: SAME
inq FE: SAME
member: SAME
member FE: SAME
trade: SAME
trade FE: SAME

CLIENT: bestegg
MRM: True location: /d/shared/silver_projects_v2/bestegg/personalloanv1/modeling/model_artifacts/personalloan/exp6.7.CO.pre_test.all_features_with2017_new_weights_foldinvalid/mrm_pipel

# Best Egg Looking at

In [86]:
def load_obj(folder, substring):
    location = f'{folder}{substring}'
    with open(location, 'rb') as f:
        meta_data = pickle.load(f)
        pipe = pickle.load(f)
    return pipe

In [114]:
import pandas as pd

In [106]:
path_final_CO36best_egg_pretest_folded_in =  '/d/shared/silver_projects_v2/bestegg/personalloanv1/modeling/model_artifacts/personalloan/exp7.6.CO36m.pre_test.all_features_foldinvalid/'
path_final_CO36best_egg_pretest_no_pretest_folded_in = '/d/shared/silver_projects_v2/bestegg/personalloanv1/modeling/model_artifacts/personalloan/exp7.5.CO36m.pre_test.all_features_foldinvalid/'
path_final_LGD_best_egg =  '/d/shared/silver_projects_v2/bestegg/personalloanv1/modeling/model_artifacts/personalloan/lgd_model/exp2/'

In [ ]:
ls  /d/shared/silver_projects_v2/bestegg/personalloanv1/modeling/model_artifacts/personalloan/exp7.5.CO36m.pre_test.all_features_foldinvalid/test_scores.parquet

app/                                    target/
asset.json                              test_app.parquet
best_model_params.json                  test_auc.json
calibration_object.obj                  test_data_summary.json
data_description.json                   test_fe_data.parquet
feature_definition.parquet              test_ks.json
feature_importance.parquet              test_scores.parquet
fe_data/                                test_target.parquet
fit_time_info.json                      test_zest_scores.parquet
keep_features.json                      top_features.parquet
key_factors_mapping.json                train_app.parquet
model.obj                               train_auc.json
model_strategy.json                     train_data_summary.json
mrm_pipeline.obj                        train_fe_data.parquet
overall_scores/                         train_history.json
parsed_monotonic_constraints_list.json  train_ks.json
pipeline.obj                            train_scores.parquet
run_t

In [107]:
import json 
recalibration_mapping = json.load(open(f'{path_final_CO36best_egg_pretest_no_pretest_folded_in}score_recalibration_mapping.json'))
best_params_lgd = json.load(open(f'/d/shared/silver_projects_v2/bestegg/personalloanv1/modeling/model_artifacts/personalloan/lgd_model/exp2/hyperopt_full_features_best_params.json'))
pipe_co36_best_egg = load_pipe(folder =path_final_CO36best_egg_pretest_no_pretest_folded_in)
model_lgd_bestegg = load_obj(folder =path_final_LGD_best_egg, substring = 'model.obj')
calibration_object_co36_best_egg = load_obj(folder = path_final_CO36best_egg_pretest_no_pretest_folded_in, substring = 'calibration_object.obj')

MRM: False location: /d/shared/silver_projects_v2/bestegg/personalloanv1/modeling/model_artifacts/personalloan/exp7.5.CO36m.pre_test.all_features_foldinvalid/pipeline.obj


In [ ]:
# GOing to tst 

# Following Notebook [Here](https://github.com/Katlean/client-project-bestegg/blob/master/personalloan/modeling/all_features/LGD_v3/generate_lgd_evaluation_metrics_36m.ipynb)

## Load the Data Split

In [109]:
data_split =  {
            "train": {
                "start_date": "2021-12-01",
                "end_date": "2024-02-28"
            },
            "valid":{
                "start_date": "2024-03-01",
                "end_date":"2024-03-31"
            },
            "test": {
                "start_date": "2024-04-01",
                "end_date":"2024-06-30"
            },
            "pre_test": {
                "start_date": "2021-10-01",
                "end_date":"2021-11-30"
            }
}

In [111]:
pred_loc = ['lgd_36m_exp1_lessthan_2','lgd_36m_exp4_all_vals_nocap',
           'lgd_36m_exp7_co_ever_preds_feature']
pred_location = "/d/shared/silver_projects_v2/bestegg/personalloanv1/shared_data/experiments/lgd_results_all_features_v3"

In [120]:
pre_test_lgd_scores = pd.read_parquet(f'{pred_location}/lgd_36m_exp1_lessthan_2/pre_test_preds.parquet').reset_index()
pre_test_co_scores = pd.read_parquet('/d/shared/silver_projects_v2/bestegg/personalloanv1/modeling/model_artifacts/personalloan/exp7.5.CO36m.pre_test.all_features_foldinvalid/test_scores.parquet').reset_index()

In [121]:
both_scores = pre_test_lgd_scores.merge(pre_test_co_scores, on = 'ZEST_KEY', how ='inner')

In [122]:
both_scores

,ZEST_KEY,lgd_prediction,index,final_model_predictions
0,0000000039931904491_2021-10-20_11714019,0.592238,0,0.189427
1,0000000039934009959_2021-10-13_11714021,0.764690,1,0.653476
2,0000000026545912491_2021-10-04_11714022,0.570714,2,0.106676
3,0000000021268362350_2021-11-01_11714026,0.610494,3,0.265985
4,0000000007972183416_2021-10-08_11714029,0.601549,4,0.521855
...,...,...,...,...
315906,0000000051285628439_2021-10-01_12950904,0.734481,315906,0.452883
315907,0000000045986581272_2021-10-27_12950911,0.538816,315907,0.331509
315908,0000000014373509531_2021-11-19_12950913,0.674632,315908,0.660977
315909,0000000048893841227_2021-10-22_12950919,0.691365,315909,0.592815


In [118]:
pre_test_co_scores

,ZEST_KEY,final_model_predictions
0,0000000039931904491_2021-10-20_11714019,0.189427
1,0000000039934009959_2021-10-13_11714021,0.653476
2,0000000026545912491_2021-10-04_11714022,0.106676
3,0000000021268362350_2021-11-01_11714026,0.265985
4,0000000007972183416_2021-10-08_11714029,0.521855
...,...,...
315906,0000000051285628439_2021-10-01_12950904,0.452883
315907,0000000045986581272_2021-10-27_12950911,0.331509
315908,0000000014373509531_2021-11-19_12950913,0.660977
315909,0000000048893841227_2021-10-22_12950919,0.592815


In [ ]:
for model_name in pred_loc:
    print(f"Processing: {model_name}")

    pred_df = pd.read_parquet(pred_location+'/'+model_name+'/pre_test_preds.parquet')

# Load and Prepare Pre-Test Target the Same Way

In [131]:
mkdir /d/shared/users/jag/truist/

In [132]:
mkdir /d/shared/users/jag/truist/jackson_experiment/

In [133]:
mkdir /d/shared/users/jag/truist/jackson_experiment/test

# LOAD DATA

In [ ]:
eval_output_folder = '/d/shared/users/jag/truist/jackson_experiment/test'
pred_location = "/d/shared/silver_projects_v2/bestegg/personalloanv1/shared_data/experiments/lgd_results_all_features_v3"
pre_test_all_unit_path = '/d/shared/silver_projects_v2/bestegg/personalloanv1/shared_data/processed_data/all_features/lgd_modeling_dataset_v2/lgd_all_data_ageatdefault_v2.parquet'
pre_test_all_data_path = '/d/shared/silver_projects_v2/bestegg/personalloanv1/shared_data/processed_data/all_features/PL_DOLLAR_LOSS_PERF_2021Q4_20251210_hit_processed.parquet'
pre_test_all_data = pd.read_parquet(pre_test_all_data_path)
pre_test_all_unit = pd.read_parquet(pre_test_all_unit_path)
pre_test_all_data['APP_DATE'] = pd.to_datetime(pre_test_all_data['APP_DATE'])



: 

In [ ]:
pre_test_all_data.shape, pre_test_all_unit.shape

# FIRST FILTERING THE PRETEST ALL DATA 

In [ ]:
pre_test_all_data = pre_test_all_data[(pre_test_all_data['APP_DATE'] >= data_split['pre_test']['start_date']) & (pre_test_all_data['APP_DATE'] <= data_split['pre_test']['end_date'])]

In [124]:
target_pre_test = pd.read_parquet('/d/shared/silver_projects_v2/bestegg/personalloanv1/shared_data/processed_data/all_features/CO_targets_proxy_v2/PL_DOLLAR_LOSS_PERF_2021Q4_20251210_hit_processed_proxy_target.parquet')

In [ ]:
pre_test_all_unit_path = '/d/shared/silver_projects_v2/bestegg/personalloanv1/shared_data/processed_data/all_features/lgd_modeling_dataset_v2/lgd_all_data_ageatdefault_v2.parquet'

KeyError: 'unit_target'